In [ ]:
#heheheh
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

path_train = '/kaggle/input/melanoma-cancer-dataset/train/'
path_test = '/kaggle/input/melanoma-cancer-dataset/test/'

In [ ]:
import os 
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks

epochs = 100
batch_size = 32 # tamanho da separação do sgd, cada vez que roda ele atualiza os pessos da rede, entt ele vai atualizar os pesos 32 vezes, mtt pequeno é mais rápido mas instável
patience = 5 # quanto que o earlystop vai esperar para para, nesse caso são 5 epochs
learning_rate = 0.001 # alfa do SGD
model_path = 'kaggle/checkpoints/model.keras' # onde ele vai salvar o modelo, o 'dump'

In [ ]:
model = models.Sequential([
    # diminuir para fazer covuluções sem dar milhões de parâmetros
    layers.Resizing(56, 56), # colocando todas as imagens do mesmo tamanho, nn corta a imagem apenas diminui a resolução    
    layers.Rescaling(1.0/255), # deixar todos os valores dos pixels entre 0 e 1
    layers.RandomRotation((-0.2 , 0.2)), # leve rotação para cada imagem, rotacionar deixa mais lento mas aumenta a precisão


    # covolucao
    layers.Conv2D(32, (3, 3), # 32 filtros 3 por 3, tenho 32 imagens 54 por 54
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal() # valor aleatório com distribuição normal 
    ),
    
    layers.MaxPooling2D((2, 2)), # 32 imagens 27 por 27, pooling pega cada 4 quadradinhos e pega o valor máximo de cada quadradinho, isso diminui a lentidão

    layers.Conv2D(64, (4, 4),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
                  
    layers.MaxPooling2D((2, 2)),
                  
    layers.Flatten(), # vai pegar tds as imagens 9 x 8 e tranformar num vetor gigante

    layers.Dropout(0.2), # desativa 20% dos dados na camada
                  
    layers.Dense(128, 
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
                  
    layers.Dense(64, 
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(1, 
        activation = 'sigmoid',
        kernel_initializer = initializers.RandomNormal()
    )
])

In [ ]:
model.compile(
    optimizer = optimizers.Adam( # adam é um algoritmo keras
        learning_rate = learning_rate
    ),
    loss = losses.BinaryCrossentropy(),
    metrics = [metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall() ]
)

In [ ]:
train = utils.image_dataset_from_directory(
    path_train,
    # validation_split = 0.2,
    # subset= 'training',
    shuffle=True,
    seed=42,
    image_size=(244,244),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    path_test,
    # validation_split = 0.2,
    # subset= 'validation',
    shuffle=True,
    seed=42,
    image_size=(244,244),
    batch_size = batch_size
)

In [ ]:
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience
        )
        # callbacks.ModelCheckpoint(
        #     filepath = model_path,
        #     save_weights_only = False,
        #     monitor = 'loss',
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)